In [11]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import time
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn import metrics
from datetime import datetime
from IPython.display import HTML
from matplotlib import animation

# Ignore some warnings produced by basemap
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# try to import Basemap
try:
    from mpl_toolkits.basemap import Basemap
    has_basemap = True
except ImportError:
    has_basemap = False

# Set the map resolution
# Notice: set resolution to full will make this program time-consuming
map_resolution = "i" # l: low, i: intermediate, h: high, f: full
    
plt.rcParams['figure.figsize'] = (20,5) # fiqure size

In [12]:
with open('201707-citibike-tripdata.csv', mode='r', encoding='utf-8') as file:
    trip_data = pd.read_csv(file)
    trip_data.columns = trip_data.columns.str.replace('\s+', '_')

trip_data.head(100)

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2
5,545,2017-07-01 00:00:18,2017-07-01 00:09:24,361,Allen St & Hester St,40.716059,-73.991908,502,Henry St & Grand St,40.714215,-73.981346,21203,Subscriber,1954.0,2
6,1496,2017-07-01 00:00:18,2017-07-01 00:25:15,3233,E 48 St & 5 Ave,40.757246,-73.978059,546,E 30 St & Park Ave S,40.744449,-73.983035,15933,Customer,1971.0,1
7,1495,2017-07-01 00:00:19,2017-07-01 00:25:15,3233,E 48 St & 5 Ave,40.757246,-73.978059,546,E 30 St & Park Ave S,40.744449,-73.983035,18515,Customer,1964.0,2
8,593,2017-07-01 00:00:27,2017-07-01 00:10:20,3314,W 95 St & Broadway,40.793770,-73.971888,3363,E 102 St & Park Ave,40.790483,-73.950331,29135,Subscriber,1982.0,1
9,1950,2017-07-01 00:00:30,2017-07-01 00:33:00,3085,Roebling St & N 4 St,40.714690,-73.957390,468,Broadway & W 56 St,40.765265,-73.981923,17365,Subscriber,1983.0,1


In [8]:
with open('201707-citibike-tripdata.csv', mode='r', encoding='utf-8') as file:
    trip_data = pd.read_csv(file, parse_dates=['starttime', 'stoptime'])
    trip_data.columns = trip_data.columns.str.replace('\s+', '_')

trip_data.head(100)

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2
5,545,2017-07-01 00:00:18,2017-07-01 00:09:24,361,Allen St & Hester St,40.716059,-73.991908,502,Henry St & Grand St,40.714215,-73.981346,21203,Subscriber,1954.0,2
6,1496,2017-07-01 00:00:18,2017-07-01 00:25:15,3233,E 48 St & 5 Ave,40.757246,-73.978059,546,E 30 St & Park Ave S,40.744449,-73.983035,15933,Customer,1971.0,1
7,1495,2017-07-01 00:00:19,2017-07-01 00:25:15,3233,E 48 St & 5 Ave,40.757246,-73.978059,546,E 30 St & Park Ave S,40.744449,-73.983035,18515,Customer,1964.0,2
8,593,2017-07-01 00:00:27,2017-07-01 00:10:20,3314,W 95 St & Broadway,40.793770,-73.971888,3363,E 102 St & Park Ave,40.790483,-73.950331,29135,Subscriber,1982.0,1
9,1950,2017-07-01 00:00:30,2017-07-01 00:33:00,3085,Roebling St & N 4 St,40.714690,-73.957390,468,Broadway & W 56 St,40.765265,-73.981923,17365,Subscriber,1983.0,1
